In [2]:
!pip install torchmetrics

In [3]:
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from collections import Counter

import torch
import torch.nn.functional as F
from torch import nn
from torchmetrics.functional import accuracy
from torch.utils.data import Dataset, DataLoader, \
    RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt
from gensim.models import KeyedVectors

import nltk
from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
df = pd.read_csv('../input/dataminingproject2/Fake.csv')

In [7]:
def preprocess(text):
    text = text.lower()
    text = word_tokenize(text)
    return text

In [8]:
df['preprocessed_text'] = df['text'].apply(lambda x: preprocess(x))

In [9]:
import nltk.data
df['max_sent_len'] = 0
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
for i in range(len(df)):
  sents = tokenizer.tokenize(df['text'][i])
  sents_len = [len(preprocess(x)) for x in sents]
  if len(sents_len) != 0:
    df['max_sent_len'][i] = max(sents_len)
  else:
    df['max_sent_len'][i] = 0

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [10]:
df['target'] = df['subject'].astype('category').cat.codes

In [12]:
df['target'].unique()

array([2, 5, 0, 4, 3, 1], dtype=int8)

In [13]:
data = df[['preprocessed_text', 'target']]

In [14]:
train_sentences, test_sentences = train_test_split(data, test_size=0.2)

In [15]:
vocab = Counter()
for line in tqdm(df.preprocessed_text):
    vocab.update(line)

  0%|          | 0/23481 [00:00<?, ?it/s]

In [16]:
len(vocab.keys())

181384

In [17]:
filtered_vocab = set()

for word in vocab.keys():
    if vocab[word] > 2:
        filtered_vocab.add(word)

In [18]:
len(filtered_vocab)

59642

In [19]:
word2id = {'UNK': 0}

for word in filtered_vocab:
    word2id[word] = len(word2id)

In [20]:
MAX_LEN = max(df['max_sent_len'].tolist())
MAX_LEN

2243

In [21]:
! wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-12-29 19:10:16--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.134.112
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.134.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  28.3MB/s    in 50s     

2021-12-29 19:11:06 (31.4 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [23]:
import gensim

# Вставьте сюда путь до модели с В2В векторами
google_news_path = "GoogleNews-vectors-negative300.bin.gz"
w2v = gensim.models.KeyedVectors.load_word2vec_format(google_news_path, binary=True)

In [24]:
weights = np.zeros((len(word2id), 300))
for word, i in word2id.items(): 
    try:
        weights[i] = w2v[word]    
    except KeyError:
      pass

In [25]:
class MyDataset(Dataset):

    def __init__(self, dataset, word2id, DEVICE):
        self.dataset = dataset['preprocessed_text'].values
        self.word2id = word2id
        self.length = dataset.shape[0]
        self.target = torch.Tensor(dataset['target'].values)
        self.device = DEVICE

    def __len__(self):
        return self.length

    def __getitem__(self, index): 
        tokens = self.dataset[index] 
        ids = torch.LongTensor([self.word2id[token] if token in self.word2id \
                                else self.word2id['UNK'] for token in tokens][:MAX_LEN])
        y = [self.target[index]]
        return ids, y

    def collate_fn(self, batch): 
        ids, y = list(zip(*batch))
        padded_ids = torch.LongTensor(torch.stack([F.pad(idx, (1, MAX_LEN - len(idx))) for idx in ids], dim=0)).to(self.device)
        y = torch.LongTensor(y).view(len(y)).to(self.device)
        return padded_ids, y

In [26]:
print('Доступна ли видеокарта:', torch.cuda.is_available())
print('Если недоступна, поменяйте runtime, если в колабе')

Доступна ли видеокарта: True
Если недоступна, поменяйте runtime, если в колабе


In [27]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [28]:
train_dataset = MyDataset(train_sentences, word2id, DEVICE)
train_sampler = RandomSampler(train_dataset)
train_iterator = DataLoader(train_dataset, collate_fn = train_dataset.collate_fn, sampler=train_sampler, batch_size=128)

In [29]:
test_dataset = MyDataset(test_sentences, word2id, DEVICE)
test_sampler = SequentialSampler(test_dataset)
test_iterator = DataLoader(test_dataset, collate_fn = test_dataset.collate_fn, sampler=test_sampler, batch_size=128)

https://github.com/KifayatMsd/C-LSTM-text-classification/blob/master/clstm_classifier.py

In [30]:
class CNN_LSTM(nn.Module):

    def __init__(self, dropout, max_length, filter_sizes, vocab_size, 
                 filter_number, memory_dimension):
        super().__init__()
        self.max_length = max_length
        self.vocab_size = vocab_size
        self.filter_sizes = filter_sizes
        self.filter_number = filter_number
        self.memory_dimension = memory_dimension

        self.embedding = nn.Embedding(vocab_size, 300)
        self.embedding.from_pretrained(torch.tensor(weights))  
        
        self.conv2 = nn.Conv1d(in_channels=300, out_channels=filter_number, kernel_size=2)
        self.conv3 = nn.Conv1d(in_channels=300, out_channels=filter_number, kernel_size=3)
        self.conv4 = nn.Conv1d(in_channels=300, out_channels=filter_number, kernel_size=4)
        
        self.convdict = {2 : self.conv2, 3 : self.conv3, 4 : self.conv4}
        
        self.lstm = nn.LSTM(input_size=filter_number, hidden_size=memory_dimension, num_layers=1, batch_first=True)
        self.dropout = nn.Dropout(p=dropout)
              
        self.relu = nn.ReLU()
        self.out = nn.Softmax(dim=1)
        self.linear = nn.Linear(memory_dimension, 6)

    def forward(self, word):
        embedded = self.embedding(word)
        embedded = embedded.transpose(1, 2)

        max_feature_length = self.max_length - max(self.filter_sizes) + 1 
        conv_outputs = []
        for size in self.filter_sizes:
            layer = self.convdict[size]
            layer = layer(embedded)
            h = self.relu(layer)
            h = h[:, :, :max_feature_length]
            conv_outputs.append(h)
        if len(self.filter_sizes) > 1: 
            rnn_inputs = torch.cat(conv_outputs, -1)
        else:
            rnn_inputs = h
            
        _, (hidden_state, _) = self.lstm(rnn_inputs.transpose(1, 2))
        logits = self.out(self.linear(torch.squeeze(hidden_state, 0)))
        
        return logits 

In [31]:
def train(model, optimizer, criterion, epoch, metric):

    progress_bar = tqdm(total=len(train_iterator.dataset), desc='Epoch {}'.format(epoch + 1))
    epoch_losses = []
    epoch_metrics = []

    model.train()

    for i, (texts, ys) in enumerate(train_iterator):
        optimizer.zero_grad() 
        preds = model(texts)
        loss = criterion(preds, ys)  
        loss.backward() 
        optimizer.step() 
        epoch_losses.append(loss.item())
        batch_metric = metric(preds.argmax(1).long(), ys.long(),
                                  average='weighted', num_classes=6,
                                  ignore_index=0)
        epoch_metrics.append(batch_metric.cpu().numpy())

        if not (i + 1) % int(len(train_iterator)/5):
            print(f'Train loss: {np.mean(epoch_losses)}, Train metric: {np.mean(epoch_metrics)}')  
        progress_bar.update(texts.shape[0])
    return np.mean(epoch_losses), np.mean(epoch_metrics)

In [32]:
def evaluate(model, criterion, n_epoch, metric):

    epoch_losses = []
    epoch_metrics = []
    model.eval()

    with torch.no_grad():

        for i, (texts, ys) in enumerate(test_iterator):
            preds = model(texts)
            loss = criterion(preds, ys)

            epoch_losses.append(loss.item())
            batch_metric = metric(preds.argmax(1).long(), ys.long(),
                                  average='weighted', num_classes=6,
                                  ignore_index=0)
            epoch_metrics.append(batch_metric.cpu().numpy())

            if (i + 1) % int(len(test_iterator) / 5) == 0:
                print('Val loss: {}, Val metric: {}'.format(
                   epoch_losses), epoch_metrics))

    return np.mean(epoch_losses), np.mean(epoch_metrics)

In [34]:
model = CNN_LSTM(drop, MAX_LEN, filter, len(word2id), 150, 150)
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001,
                                weight_decay=0.0001)
criterion = nn.CrossEntropyLoss()

model = model.to(DEVICE)
criterion = criterion.to(DEVICE)

In [38]:
filter_variations = [[2], [3], [4], [2, 3], [3, 4], [2, 4], [2, 3, 4]]

all_results = []
train_loss = []
train_acc = []
test_loss = []
test_acc = []
for filters in filter_variations:
    for drop in [0, 0.2, 0.4]: 
        print('Training new model: filter=', filters, ', drop=', drop)
        model = CNN_LSTM(drop, MAX_LEN, filters, len(word2id), 150, 150)
        model = model.to(DEVICE)
        optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001,
                                weight_decay=0.0001)
        criterion = nn.CrossEntropyLoss()
        criterion = criterion.to(DEVICE)
        epochs = 3
        for i_epoch in range(epochs):
            print('Training...)))')
            epoch_losses, epoch_metrics = train(model, optimizer, criterion, i_epoch, accuracy)
            train_loss.append(epoch_losses)
            train_acc.append(epoch_metrics)
            print('Validating...)))')
            epoch_losses_test, epoch_metrics_test = evaluate(model, criterion, i_epoch, accuracy)
            test_loss.append(epoch_losses_test)
            test_acc.append(epoch_metrics_test)
        all_results.append({'filter': filters, 'dropout': drop, 
                            'train_loss': train_loss, 'train_acc': train_acc,
                            'test_loss': test_loss, 'test_acc': test_acc})

Training new model: filter= [2] , drop= 0
Training...)))


Epoch 1:   0%|          | 0/18784 [00:00<?, ?it/s]

Train loss: 1.6531748977200738, Train metric: 0.4032786190509796
Train loss: 1.6464090614483273, Train metric: 0.40610745549201965
Train loss: 1.646457548799186, Train metric: 0.4064714312553406
Train loss: 1.643988208524112, Train metric: 0.4098382294178009
Train loss: 1.6427173096558143, Train metric: 0.41016244888305664
Validating...)))
Val loss: 1.6301256588527135, Val metric: 0.43471115827560425
Val loss: 1.635133249419076, Val metric: 0.42621782422065735
Val loss: 1.6366019192196073, Val metric: 0.4234215319156647
Val loss: 1.6366915149348122, Val metric: 0.4245997965335846
Val loss: 1.6396667787006922, Val metric: 0.41894587874412537
Training...)))


Epoch 2:   0%|          | 0/18784 [00:00<?, ?it/s]

Train loss: 1.6347932651125152, Train metric: 0.41494372487068176
Train loss: 1.6365536369126419, Train metric: 0.41479650139808655
Train loss: 1.6372406701932007, Train metric: 0.4145984351634979
Train loss: 1.6375939640505561, Train metric: 0.41565194725990295
Train loss: 1.638358759880066, Train metric: 0.41491347551345825
Validating...)))
Val loss: 1.6300863368170602, Val metric: 0.43471115827560425
Val loss: 1.6350059764725822, Val metric: 0.42621782422065735
Val loss: 1.6365070853914534, Val metric: 0.4234215319156647
Val loss: 1.6370084498609816, Val metric: 0.4245997965335846
Val loss: 1.6396270411355154, Val metric: 0.41894587874412537
Training...)))


...

Train loss: 1.6540122730978604, Train metric: 0.38580062985420227
Train loss: 1.6478489164648384, Train metric: 0.39893823862075806
Train loss: 1.6451661627868126, Train metric: 0.4047055244445801
Train loss: 1.643900854834195, Train metric: 0.40557733178138733
Train loss: 1.6423470135392815, Train metric: 0.40710797905921936
Validating...)))
Val loss: 1.6142224414008004, Val metric: 0.43471115827560425
Val loss: 1.6196935176849365, Val metric: 0.42621782422065735
Val loss: 1.621407111485799, Val metric: 0.4234215319156647
Val loss: 1.6217106410435267, Val metric: 0.4245997965335846
Val loss: 1.6243730987821305, Val metric: 0.41894587874412537
Training...)))


Epoch 2:   0%|          | 0/18784 [00:00<?, ?it/s]

Train loss: 1.5848799573964085, Train metric: 0.5018948316574097
Train loss: 1.5565659527120919, Train metric: 0.5971381664276123
Train loss: 1.5375456631868736, Train metric: 0.6325547099113464
Train loss: 1.5197950807111016, Train metric: 0.6531293392181396
Train loss: 1.5026289989208352, Train metric: 0.6647008657455444
Validating...)))
Val loss: 1.4066561630793981, Val metric: 0.7379445433616638
Val loss: 1.412551760673523, Val metric: 0.7295905351638794
Val loss: 1.4157694748469762, Val metric: 0.7233187556266785
Val loss: 1.4175427598612649, Val metric: 0.7200733423233032
Val loss: 1.4195818287985666, Val metric: 0.7175746560096741
Training...)))


Epoch 3:   0%|          | 0/18784 [00:00<?, ?it/s]

Train loss: 1.40636539048162, Train metric: 0.7154887914657593
Train loss: 1.3926019319172562, Train metric: 0.721647322177887
Train loss: 1.3871343026216003, Train metric: 0.720188558101654
Train loss: 1.3853263382253975, Train metric: 0.7194879055023193
Train loss: 1.383189476769546, Train metric: 0.7187732458114624
Validating...)))
Val loss: 1.3643612691334315, Val metric: 0.7367540597915649
Val loss: 1.370169767311641, Val metric: 0.7302058935165405
Val loss: 1.3734106847218104, Val metric: 0.7237185835838318
Val loss: 1.3752629884651728, Val metric: 0.7206708788871765
Val loss: 1.3773105314799718, Val metric: 0.7175663709640503


Фильтр [2,3,4] показывает неплохие скоры, но в целом результаты модели не очень, потому что в датасете дисбаланс классов.